In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from selenium.common.exceptions import StaleElementReferenceException
import requests
from datetime import datetime
from selenium.webdriver.firefox.options import Options

df = pd.DataFrame(columns=["Session", "Names"])

# Get links for each congress and get corresponding congress numbers
links = [f"https://www.c-span.org/congress/members/?congress={i}&congressyear&chamber=house&visual&find-name=all&find-state=all&find-party=all&status=all&sort-names=name&all" for i in range(80, 118)]
sessions = [i for i in range(80, 118)]
data = []

options = Options()
options.headless = True 

# Use Firefox WebDriver since Chrome updated
driver = webdriver.Firefox(options=options)

for session, link in zip(sessions, links):
    driver.get(link)
    
    # Wait for page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    # Get all senator names
    senators = driver.find_elements(By.CSS_SELECTOR, "a.name")
    # Get the senator's states and political parties
    states_and_parties = driver.find_elements(By.XPATH, '//span[@class="organization"]')

    # Loop through each senator's info
    for senator, party_state in zip(senators, states_and_parties):
        name = senator.find_element(By.CSS_SELECTOR, "h3").text
        text = party_state.text

        if text.startswith('R'):
            party = 'Republican'
        elif text.startswith('D'):
            party = 'Democrat'
        else:
            party = 'Old Party'
        state = text[2:]
        # Do slicing since format is like: 'R-OH', so to get state we don't want first two letters

        # Add each senator's info as new row in the DataFrame that corresponds to a given senator and given session of congress
        data.append({"Session": session, "Name": name, "Party": party, "State": state})
        print(name, ": ", party)

    print(f"Session {session} successfully processed")
df = pd.DataFrame(data)
df.to_csv("C:/Work/EconResearch/Files/RepresentativesByCongress.csv", index=False)
driver.quit()

WATKINS MOORMAN ABBITT :  Democrat
THOMAS GERSTLE ABERNETHY :  Democrat
Carl Albert :  Democrat
LEO ELWOOD ALLEN :  Republican
ASA LEONARD ALLEN :  Democrat
JOHN JOSEPH ALLEN :  Republican
JAMES LINDSAY ALMOND :  Democrat
HERMAN CARL ANDERSEN :  Republican
JOHN ZUINGLIUS ANDERSON :  Republican
AUGUST HERMAN ANDRESEN :  Republican
GEORGE WILLIAM ANDREWS :  Democrat
WALTER GRESHAM ANDREWS :  Republican
HOMER DANIEL ANGELL :  Republican
LESLIE CORNELIUS ARENDS :  Republican
SAMUEL WASHINGTON ARNOLD :  Republican
JAMES COATS AUCHINCLOSS :  Republican
CLAUDE IGNATIUS BAKEWELL :  Republican
PARKE MONROE BANTA :  Republican
GRAHAM ARTHUR BARDEN :  Democrat
FRANK A. BARRETT :  Republican
Joe B. Bates :  Democrat
GEORGE JOSEPH BATES :  Republican
LAURIE CALVIN BATTLE :  Democrat
JAMES GLENN BEALL :  Republican
LINDLEY GARY BECKWORTH :  Democrat
CHARLES JASPER BELL :  Democrat
GEORGE HARRISON BENDER :  Republican
MARION TINSLEY BENNETT :  Republican
JOHN BONIFAS BENNETT :  Republican
Lloyd Bents

In [ ]:
# Merges our list of senators by congress with our senator biographical data

import pandas as pd

c_span = pd.read_csv("C:/Work/EconResearch/Files/SenatorsByCongress.csv")
all_data_df = pd.read_excel("C:/Work/EconResearch/Files/SenatorsAllData.xlsx")

# Columns to add to new dataframe
new_columns = [
    "Middle", "Suffix", "Nickname", "Born", "Death", "Sex", "Position", "Party",
    "State", "District", "Start", "End", "Religion", "Race", "Educational Attainment",
    "JobType1", "JobType2", "JobType3", "JobType4", "JobType5", "Mil1", "Mil2", "Mil3"
]

# Loop through each row in info_df
for _, row in all_data_df.iterrows():
    # Get first and last name
    last_name = row['Last']
    first_name = row['First']
    nickname = str(row['Nickname']) if not pd.isna(row['Nickname']) else ""
    last_name = last_name.lower().strip()
    first_name = first_name.lower().strip()
    nickname = nickname.lower().strip()
    c_span['Name'] = c_span['Name'].str.lower().str.strip()

    # To combine the dataframes, we want to match senators by last name
    # Matching by last names AND first names is tough because of nicknames in one dataset and full names in the other, leading to discrepancies
    # However, because last names can be common, in the case of first names being unable to be matched,
    # we want to have another attribute to merge them by
    # So, we make this other attribute the senator's political party and state
    # So, we can match senators between the two dataframes if they have the same first and last name OR if they have
    # the same last name as well as the same state and political party
    # In theory there could be two different senators with the same state, political party, and last name
    # But we have a finite amount of senators and so this is easily checkable to make sure it can uniquely identify senators

    # This mask will 'filter' rows of the dataframe using the conditions we just established above
    # It follows the same logic, either first and last names match OR last name matches and state and party do as well
    mask = c_span['Name'].str.contains(last_name) & (c_span['State'].str[:2] == row['State']) & ((c_span['Name'].str.contains(first_name)) | (c_span['Name'].str.contains(nickname)))
    
    # ((final_df['Party'] == row['Party']) & 
    # Update columns in final_df where mask is True (senators are able to be matched)
    for col in new_columns:
        c_span.loc[mask, col] = row[col]

# After updating matched senators, fill any remaining empty values with "NA" to make it easier to check for senators that couldn't be matched
c_span[new_columns] = c_span[new_columns].fillna("NA")

c_span.to_csv("C:/Work/EconResearch/Files/SenatorsByCongressAllData2.csv", index=False)





C:\Users\Ahiles\AppData\Local\Temp\ipykernel_13076\412796535.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Hempell' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  c_span.loc[mask, col] = row[col]
C:\Users\Ahiles\AppData\Local\Temp\ipykernel_13076\412796535.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Education' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  c_span.loc[mask, col] = row[col]
C:\Users\Ahiles\AppData\Local\Temp\ipykernel_13076\412796535.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Law' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  c_span.loc[mask, col] = row[col]
C:\Users\Ahi

In [10]:
import pandas as pd

# Checks how many senators/representatives we got in a single year
# Kinda just like basic exploratory data analysis


# Below, we just create a dictionary to check how many senators/representatives we are able to 'match' between datsets
# The key is the congress and value is the number of matches we have
# By match, I mean how many politicians we could get biographical info for

final_df = pd.read_csv("C:/Work/EconResearch/Files/SenatorsByCongressAllData2.csv")
noInfoPeople = []
final_congress_count = {}
for index, row in final_df.iterrows():
    if pd.isna(row['Born']) or row['Born'] == "NA":
        noInfoPeople.append({"Session": row['Session'], "Name": row['Name']})
    else:
        if row['Session'] not in final_congress_count:
            final_congress_count[row['Session']] = 1
        else:
            final_congress_count[row['Session']] += 1
print(final_congress_count)
noInfoDf = pd.DataFrame(noInfoPeople)
noInfoDf.to_csv("C:/Work/EconResearch/Files/MissingSenators.csv")

{80: 100, 81: 109, 82: 100, 83: 111, 84: 99, 85: 102, 86: 103, 87: 105, 88: 103, 89: 103, 90: 101, 91: 106, 92: 101, 93: 101, 94: 100, 95: 107, 96: 101, 97: 101, 98: 101, 99: 102, 100: 102, 101: 103, 102: 105, 103: 101, 104: 100, 105: 98, 106: 102, 107: 101, 108: 100, 109: 101, 110: 102, 111: 110, 112: 103, 113: 104, 114: 100, 115: 109, 116: 102, 117: 101}


In [11]:
# Merge the two datasets together

import pandas as pd

senators = pd.read_csv("C:/Work/EconResearch/Files/SenatorsByCongressAllData2.csv")
demographics = pd.read_excel("C:/Work/EconResearch/Files/DemographicData.xlsx")
demographics = demographics[demographics['State'] != 'District of Columbia']
senators = senators[senators["Session"].isin([116, 117])]
#print(demographics.columns)

session_dict = {2020: 116, 2021: 117, 2022: 117}
state_to_abbreviation = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

demographics['State'] = demographics['State'].map(state_to_abbreviation)
#demographics['Year'] = demographics['Year'].map(session_dict)


# left join keeps all rows in the df on the left and merges 
temp = pd.DataFrame()
years = [i for i in range(2020, 2023)]
temp = pd.DataFrame()
for year in years:
    # Make sure we have demographic data for a given year
    df1 = demographics[demographics['Year'] == year]
    # Make sure we only have senators for a given year
    # We get senators for a given year by filtering senators by that year's corresponding session
    # Note that mapping years to sessions is not one to one mapping
    year_session = session_dict.get(year, None)
    df2 = senators[senators['Session'] == year_session]
    result = pd.merge(df1, df2, on='State', how='left')
    temp = pd.concat([temp, result])

temp.to_csv("C:/Work/EconResearch/Files/FinalSenators.csv")
